In [20]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

## sample

In [ ]:
data = pd.read_csv("kafka_log_sample.csv", names=["Time", "user_id", "text"])
data_watch = data.loc[data.text.str.startswith("GET /data/")]
data_rate = data.loc[data.text.str.startswith("GET /rate/")]

In [ ]:
def get_rate(x):
    tmp = x["text"].split("/")[-1].split("=")
    x["movie_id"] = tmp[0]
    x["rate"] = tmp[1]
    return x

data_rate = data_rate.apply(get_rate, axis=1)

In [ ]:
np.savetxt("movie_id_sample.txt", data_rate.movie_id.unique(), fmt="%s")
np.savetxt("user_id_sample.txt", data_rate.user_id.unique(), fmt="%d")

## data_v1

In [2]:
data_rate = pd.DataFrame()

for f in os.listdir("data_0210/"):
    data = pd.read_csv("data_0210/"+f, names=["Time", "user_id", "text"])
    data_rate_tmp = data.loc[data.text.str.startswith("GET /rate/")]
    data_rate = pd.concat([data_rate, data_rate_tmp], ignore_index=True)
    print(data_rate.shape)

(6895, 3)
(13940, 3)
(20846, 3)
(27958, 3)
(33965, 3)
(39928, 3)
(46635, 3)
(51893, 3)
(58014, 3)
(64145, 3)
(70293, 3)
(76502, 3)
(78835, 3)
(85176, 3)
(91272, 3)
(97196, 3)
(103269, 3)
(109278, 3)


In [3]:
tqdm.pandas()

def get_rate(x):
    tmp = x["text"].split("/")[-1].split("=")
    x["movie_id"] = tmp[0]
    x["rate"] = tmp[1]
    return x

data_rate = data_rate.progress_apply(get_rate, axis=1)

100%|███████████████████████████████████████████████| 109278/109278 [00:38<00:00, 2828.13it/s]


In [5]:
data_rate.to_csv("processed_data/ratings_v1.csv", index=False)

In [12]:
len(data_rate.user_id.unique())

101347

In [16]:
len(data_rate.movie_id.unique())

23034

In [15]:
# user: 101347
# movie: 23034
np.savetxt("processed_data/movie_id_v1.txt", data_rate.movie_id.unique(), fmt="%s")
np.savetxt("processed_data/user_id_v1.txt", data_rate.user_id.unique(), fmt="%d")

## data_v2: large file

In [3]:
data_rate = pd.DataFrame()

with pd.read_csv("myfile.txt", names=["Time", "user_id", "text"], chunksize=1e6) as reader:
    for chunk in reader:
        data_rate_tmp = chunk.loc[chunk.text.str.startswith("GET /rate/")]
        data_rate = pd.concat([data_rate, data_rate_tmp], ignore_index=True)
        print(data_rate.shape)

(6760, 3)
(13425, 3)
(20178, 3)
(26912, 3)
(33823, 3)
(40983, 3)
(47943, 3)
(54759, 3)
(61872, 3)
(68822, 3)
(75596, 3)
(82666, 3)
(89525, 3)
(96520, 3)
(103401, 3)
(110568, 3)
(117727, 3)
(124905, 3)
(132182, 3)
(139326, 3)
(146439, 3)
(153492, 3)
(160430, 3)
(167555, 3)
(174609, 3)
(181384, 3)
(188242, 3)
(195070, 3)
(201598, 3)
(207996, 3)
(214240, 3)
(220546, 3)
(226655, 3)
(232656, 3)
(238695, 3)
(244617, 3)
(250696, 3)
(256753, 3)
(262754, 3)
(268632, 3)
(274634, 3)
(280623, 3)
(286489, 3)
(292338, 3)
(298329, 3)
(304291, 3)
(310223, 3)
(316301, 3)
(322387, 3)
(328534, 3)
(334738, 3)
(340920, 3)
(347031, 3)
(353221, 3)
(359542, 3)
(365693, 3)
(371912, 3)
(378172, 3)
(384566, 3)
(390841, 3)
(397071, 3)
(403395, 3)
(409773, 3)
(416266, 3)
(422786, 3)
(429244, 3)
(435764, 3)
(442447, 3)
(448989, 3)
(455501, 3)
(462092, 3)
(468754, 3)
(475352, 3)
(481970, 3)
(488662, 3)
(495326, 3)
(501977, 3)
(508784, 3)
(515505, 3)
(522410, 3)
(529284, 3)
(536117, 3)
(542976, 3)
(549812, 3)
(556908

ParserError: Error tokenizing data. C error: Expected 3 fields in line 254256504, saw 6


In [4]:
data_rate

,Time,user_id,text
0,2023-02-06T00:52:26,843858,GET /rate/nukes+in+space+1999=4
1,2023-02-06T00:52:26,814179,GET /rate/mulan+1998=4
2,2023-02-06T00:52:26,827266,GET /rate/my+neighbor+totoro+1988=4
3,2023-02-06T00:52:26,474885,GET /rate/the+collector+1965=4
4,2023-02-06T00:52:26,21268,GET /rate/the+terminator+1984=4
...,...,...,...
1666375,2023-02-09T10:07,234902,GET /rate/jinxed+1982=4
1666376,2023-02-09T10:07,12326,GET /rate/ss+camp+5+womens+hell+1977=3
1666377,2023-02-09T10:07,684654,GET /rate/state+fair+1933=3
1666378,2023-02-09T10:07:01,692813,GET /rate/mikes+new+car+2002=5


In [5]:
tqdm.pandas()

def get_rate(x):
    tmp = x["text"].split("/")[-1].split("=")
    x["movie_code"] = tmp[0]
    x["rate"] = tmp[1]
    return x

data_rate = data_rate.progress_apply(get_rate, axis=1)

100%|███████████████████████████████| 1666380/1666380 [07:02<00:00, 3943.35it/s]


In [6]:
movie_ids = pd.DataFrame(data_rate.movie_code.unique(), columns=['movie_code'])
movie_ids["movie_id"] = movie_ids.index

In [17]:
user_ids = pd.DataFrame(data_rate.user_id.unique(), columns=['user_id'])

In [13]:
data_rate = pd.merge(data_rate, movie_ids)

In [15]:
len(data_rate.user_id.unique())

669155

In [16]:
len(data_rate.movie_id.unique())

26826

In [19]:
user_ids.to_csv("processed_data/user_id_v2.csv", index=False)
movie_ids.to_csv("processed_data/movie_id_v2.csv", index=False)

In [23]:
data_rate[:1000000].to_csv("processed_data_v2/ratings_v2_1.csv", index=False)

In [24]:
data_rate[1000000:].to_csv("processed_data_v2/ratings_v2_2.csv", index=False)